---

_This notebook is based on materials from Coursera course [Introduction to data science with python](https://www.coursera.org/learn/python-data-analysis/)._

---

# Lab 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [37]:
import pandas as pd

df = pd.read_csv('D:/projects_git/DS_KNU/data/olympics.csv', index_col=0, skiprows=1)

for colume in df.columns:
    if colume[:2]=='01':
        df.rename(columns={colume:'Gold'+colume[4:]}, inplace=True)
    if colume[:2]=='02':
        df.rename(columns={colume:'Silver'+colume[4:]}, inplace=True)
    if colume[:2]=='03':
        df.rename(columns={colume:'Bronze'+colume[4:]}, inplace=True)
    if colume[:1]=='№':
        df.rename(columns={colume:'#'+colume[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # bc split the index by '('

df.index = names_ids.str[0] 
df['ID'] = names_ids.str[1].str[:3] 

df = df.drop('Totals')
df.head()


,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [44]:
def answer_0():
        return df.iloc[0] #bc 1st = 0

print ('Answer:')
print(answer_0())

Answer:
# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
difference          0
Name: Afghanistan, dtype: object


### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [43]:
def answer_1():
    return df[df['Gold'] == max(df['Gold'])].index[0]

print ('Answer:')
print(answer_1())

Answer:
United States


### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [53]:
def answer_2():
    df['difference'] = abs(df['Gold']-df['Gold.1'])
    return df[df['difference'] == max(df['difference'])].index[0]
print ('Answer:')
print(answer_2())


Answer:
United States


### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [46]:
def answer_3():
    a = df.copy()
    a = a.drop(a[a['Gold'].isin([0])&a['Gold.1'].isin([0])].index)  
    a = a[(a['Gold.1'] != 0) & (a['Gold'] != 0)]
    #a = a.drop(a[(a['Gold.1'] == 0) & (a['Gold'] == 0)].index) 
    #a = a.drop(a[(a['Gold'] & a['Gold.1']).isin([0])].index)
    #a = a.drop(a[a['Gold.2'] == 0].index) 
    a['relative'] = abs((a['Gold']-a['Gold.1'])/a['Gold.2'])
    return a[a['relative'] == max(a['relative'])].index[0]
print ('Answer:')
print (answer_3())

Answer:
Bulgaria


### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [48]:
def answer_4():
    return pd.Series((df['Gold.2']*3)+(df['Silver.2']*2)+df['Bronze.2'],index=df.index)
print ('Answer:')
print (answer_4())

Answer:
Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Length: 146, dtype: int64


## Part 2
For the next set of questions, we will be using census data from the **United States Census Bureau**. Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www.dropbox.com/s/a0c2fdkaxokk2ys/CO-EST2015-alldata.pdf?dl=0) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*


In [49]:
import pandas as pd
census_df = pd.read_csv('D:/projects_git/DS_KNU/data/census.csv')
census_df
def answer_5():
    state =list(census_df['STNAME'].unique())
    counts = dict()
    for item in state:
        counts[item] = census_df.where(census_df['STNAME'] == item).count()[1]
    bigcount = None
    bigword = None
    for word,count in counts.items():
        if bigcount is None or count > bigcount:
            bigword = word
            bigcount = count
    return bigword
print ('Answer:')
print (answer_5())

Answer:
Texas


### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [50]:
def answer_6():
    census_copy_df=census_df.drop(census_df[census_df['COUNTY']==0].index)
    state =list(census_copy_df['STNAME'].unique())
    counts = dict()
    for item in state:
        counts[item] = sum(census_copy_df.where(census_copy_df['STNAME'] == item).nlargest(3,'CENSUS2010POP')['CENSUS2010POP'])
    return list(pd.Series(counts).nlargest(3).index)
print ('Answer:')
print (answer_6())

Answer:
['California', 'Texas', 'Illinois']


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [51]:
def answer_7():
    census_copy=census_df.drop(census_df[census_df['COUNTY']==0].index)
    census_copy['max_dif']=census_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].max(axis=1)-census_copy[['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']].min(axis=1)
    return list(census_copy[census_copy['max_dif'] == max(census_copy['max_dif'])]['CTYNAME'])[0]  #.astype(str)
print ('Answer:')
print (answer_7())

Answer:
Harris County


### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [52]:
def answer_8():
    region = census_df.where((census_df['REGION'] == 1) | (census_df['REGION'] == 2)).dropna()
    region = region[region['CTYNAME'].str.startswith('Washington')]
    region= region[region['POPESTIMATE2015'] > region['POPESTIMATE2014']]
    return region[['STNAME', 'CTYNAME']]
print ('Answer:')
print (answer_8())

Answer:
            STNAME            CTYNAME
896           Iowa  Washington County
1419     Minnesota  Washington County
2345  Pennsylvania  Washington County
2355  Rhode Island  Washington County
3163     Wisconsin  Washington County
